In [ ]:
import socket
import sys

is_kaggle = True
if not is_kaggle:
    from my_utils import get_notebook_path
    get_notebook_path().split('/')[-1].split('.')[0]
    NB = get_notebook_path().split('/')[-1].split('.')[0]
    HOST = socket.gethostname()
    import mlflow
    from logzero import logger
    DELIMITER = '\\'

else:
    NB = 'exp0093'
    HOST = 'DESKTOP-M3SEAIN'
    sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
    DELIMITER = '/'
description = '256'

print(f'{HOST}_{NB}')

NB, HOST

In [ ]:
#from cosine_annealing_warmup import CosineAnnealingWarmupRestarts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
from sklearn.model_selection import StratifiedKFold, KFold
if not is_kaggle:
    from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error
from pathlib import Path
from tqdm import tqdm
from collections import OrderedDict
import random
import os
import gc
import shutil
import timm


import torch.optim as optim
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import albumentations as albu

import pickle

import warnings
warnings.simplefilter('ignore')

if is_kaggle:
    DATA_DIR = Path('../input/petfinder-pawpularity-score')
    OUTPUT_DIR = Path('./')
    CP_DIR = Path('../input') / Path('DESKTOP-M3SEAIN_exp0093'.lower().replace('-', '').replace('_', '-'))
else:
    ROOT_DIR = Path('../../')
    DATA_DIR = ROOT_DIR / 'data'
    OUTPUT_DIR = Path('../') / 'output'
    CP_DIR = OUTPUT_DIR / 'checkpoint'

def to_pickle(filename, obj):
    with open(filename, mode='wb') as f:
        pickle.dump(obj, f)
        
def unpickle(filename):
    with open(filename, mode='rb') as fo:
        p = pickle.load(fo)
    return p 

In [ ]:

class Config:
    N_LABEL = 1
    N_FOLD = 5
    RANDOM_SATE = 42
    LR = 1.0e-05
    MAX_LR = 1e-5
    PATIENCE = 18
    EPOCH = 10
    BATCH_SIZE = 32
    SKIP_EVALUATE_NUM = 0
    BACK_BONE = 'swin_large_patch4_window7_224'
    RUN_FOLD_COUNT = 10
    IMG_SIZE=224
    T_MAX=20
    ETA_MIN=3.0e-7
    SCHEDULER_GAMMA=1.0
    ACCUMULATION_STEMP=1 
    
if is_kaggle:
    Config.BATCH_SIZE = 24
    
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(seed=Config.RANDOM_SATE)

In [ ]:
glob.glob('../input/*')

In [ ]:
test_img_map = {}
for img_path in tqdm(glob.glob(str(DATA_DIR /  'test/*'))):
    test_img_map[img_path.split(DELIMITER)[-1].split('.')[0]] = img_path

In [ ]:
test_df = pd.read_csv(DATA_DIR / 'test.csv')

## johannyjm_1790651

In [ ]:
from fastai.vision.all import *

In [ ]:
SEED = 46
BATCH_SIZE = 16
set_seed(SEED, reproducible=True)
dataset_path = Path('../input/petfinder-pawpularity-score/')
dataset_path.ls()

In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'

In [ ]:
set_seed(SEED, reproducible=True)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

In [ ]:
def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

In [ ]:
test_df = pd.read_csv(DATA_DIR / 'test.csv')

test_path_list = []
for f in glob.glob('../input/petfinder-pawpularity-score/test/*'):
    test_path_list.append(
        {
            'Id':f.split('/')[-1].split('.')[0],
            'path':f
        }
    )
test_df = test_df.merge(pd.DataFrame(test_path_list), how='left', on='Id')
test_df['norm_score'] = -1
test_df

In [ ]:
%%time
all_preds = []

for i in range(10):
    
    print(f'===={i}====')
    
    path = '../input/johannyjm-1790651/'
        
    learn = load_learner(path + f'model_fold_{i}.pkl', cpu=False)
        
    dls = ImageDataLoaders.from_df(test_df, #pass in train DataFrame
                               valid_pct=0.2, #80-20 train-validation random split
                               seed=SEED, #seed
                               fn_col='path', #filename/path is in the second column of the DataFrame
                               label_col='norm_score', #label is in the first column of the DataFrame
                               y_block=RegressionBlock, #The type of target
                               bs=BATCH_SIZE, #pass in batch size
                               num_workers=4,
                               item_tfms=Resize(224), #pass in item_tfms
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) 
    
    test_dl = dls.test_dl(test_df)
    
    preds, _ = learn.tta(dl=test_dl, n=5, beta=0)
    
    all_preds.append(preds)
    
    del learn, test_dl

    torch.cuda.empty_cache()

    gc.collect()

In [ ]:
y_johannyjm_1790651 = np.mean(np.stack(all_preds), axis=0).reshape(-1) * 100
y_johannyjm_1790651

## fast_ai_0004


In [ ]:
%%time
fast_ai_004_all_preds = []

for i in range(5):
    
    print(f'===={i}====')
    
    
    
    path = '../input/fastai-004/'
        
    learn = load_learner(path + f'fastai_fastai_0004_{i}.pkl', cpu=False)
    
        
    dls = ImageDataLoaders.from_df(test_df, #pass in train DataFrame
                               valid_pct=0.2, #80-20 train-validation random split
                               seed=SEED, #seed
                               fn_col='path', #filename/path is in the second column of the DataFrame
                               label_col='norm_score', #label is in the first column of the DataFrame
                               y_block=RegressionBlock, #The type of target
                               bs=BATCH_SIZE, #pass in batch size
                               num_workers=4,
                               item_tfms=Resize(224), #pass in item_tfms
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) 
    
    test_dl = dls.test_dl(test_df)
    
    preds, _ = learn.tta(dl=test_dl, n=10, beta=0)
    
    fast_ai_004_all_preds.append(preds)
    
    del learn, test_dl

    torch.cuda.empty_cache()

    gc.collect()

In [ ]:
fast_ai_004_preds = np.mean(np.stack(fast_ai_004_all_preds), axis=0).reshape(-1) * 100
fast_ai_004_preds

In [ ]:

class Config:
    N_LABEL = 1
    N_FOLD = 5
    RANDOM_SATE = 42
    LR = 1.0e-05
    MAX_LR = 1e-5
    PATIENCE = 18
    EPOCH = 10
    BATCH_SIZE = 32
    SKIP_EVALUATE_NUM = 0
    BACK_BONE = 'swin_large_patch4_window7_224'
    RUN_FOLD_COUNT = 10
    IMG_SIZE=224
    T_MAX=20
    ETA_MIN=3.0e-7
    SCHEDULER_GAMMA=1.0
    ACCUMULATION_STEMP=1 
seed_everything(seed=Config.RANDOM_SATE)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

## yuki DataAugmentations

In [ ]:
def get_test_augmentation(IM_SZ=384):
    train_transform = [
        albu.Resize(height=IM_SZ, width=IM_SZ),
        albu.Normalize(),
    ]
    return albu.Compose(train_transform)

def get_fllsize_augmentation(IM_SZ=256):
    train_transform = [
        albu.LongestMaxSize(max_size=IM_SZ, always_apply=False),
        albu.PadIfNeeded(always_apply=True, min_height=IM_SZ, min_width=IM_SZ, border_mode=2),
        albu.Normalize(),
    ]
    return albu.Compose(train_transform)

## yuki Datasets

In [ ]:
class PetfinderDataSet_base(Dataset):
    def __init__(self, df, img_map, transforms, data_type=None):
        self.df = df
        self.img_map = img_map
        self.data_type = data_type
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if self.data_type in ['train', 'valid']:
            pawpularity = self.df.iloc[idx]['Pawpularity']
            object_id = self.df.iloc[idx]['Id']
            img = self.img_map[object_id]
        else:
            pawpularity = -1
            object_id = self.df.iloc[idx]['Id']
            file_path = self.img_map[object_id]
            img = cv2.imread(file_path)
            
        height = img.shape[0]
        width = img.shape[1]
            
        if height > width:
            center = int(height/2)
            half = int(width/2)
            img = img[center - half:center + half, :]

        elif width > height:
            center = int(width/2)
            half = int(height/2)
            img = img[:, center - half:center + half]
        
        augmented = self.transforms(image=img)
        img = augmented['image']
        img = np.moveaxis(img, 2, 0)

        return img, pawpularity / 100

In [ ]:
class PetfinderDataSet_fllsize(Dataset):
    # default fllsize_transforms --> get_fllsize_augmentation(IM_SZ=256)
    def __init__(self, df, img_map, transforms, data_type=None, fllsize_transforms=get_fllsize_augmentation(IM_SZ=256)):
        self.df = df
        self.img_map = img_map
        self.data_type = data_type
        self.transforms = transforms
        self.fllsize_transforms = fllsize_transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if self.data_type in ['train', 'valid']:
            pawpularity = self.df.iloc[idx]['Pawpularity']
            object_id = self.df.iloc[idx]['Id']
            img = self.img_map[object_id]
        else:
            pawpularity = -1
            object_id = self.df.iloc[idx]['Id']
            file_path = self.img_map[object_id]
            img = cv2.imread(file_path)
            
        full_augmented = self.fllsize_transforms(image=img)
        full_img = full_augmented['image']
        full_img = np.moveaxis(full_img, 2, 0)
            
        height = img.shape[0]
        width = img.shape[1]
            
        if height > width:
            center = int(height/2)
            half = int(width/2)
            img = img[center - half:center + half, :]

        elif width > height:
            center = int(width/2)
            half = int(height/2)
            img = img[:, center - half:center + half]
        
        augmented = self.transforms(image=img)
        img = augmented['image']
        img = np.moveaxis(img, 2, 0)

        return img, full_img, pawpularity

## yuki Models

In [ ]:
class PetfinderNet(nn.Module):
    # for 0080, c0007, 0097, 0101, Age
    def __init__(self, model_name):
        super(PetfinderNet, self).__init__()

        self.base_model = timm.create_model(model_name, num_classes=0, pretrained=False, in_chans=3)
        num_features = self.base_model.num_features

        self.cls = nn.Sequential(
            nn.Linear(num_features, int(num_features / 2)),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(int(num_features / 2), int(num_features / 4)),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(int(num_features / 4), Config.N_LABEL)
        )

    def forward(self, x):
        x = self.base_model(x)
        x = self.cls(x)
        return x

In [ ]:
class PetfinderDoubleBaseNet(nn.Module):
    # for 0030, 0095
    def __init__(self):
        super(PetfinderDoubleBaseNet, self).__init__()

        self.base_model1 = timm.create_model('tf_efficientnet_b1_ns', num_classes=0, pretrained=False, in_chans=3)
        self.base_model2 = timm.create_model('tf_efficientnet_b1_ns', num_classes=0, pretrained=False, in_chans=3)
        num_features = self.base_model1.num_features

        self.cls = nn.Sequential(
            nn.Linear(num_features * 2, int(num_features)),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(int(num_features), int(num_features / 2)),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(int(num_features / 2), Config.N_LABEL)
        )

    def forward(self, x1, x2):
        x1 = self.base_model1(x1)
        x2 = self.base_model1(x2)
        
        x = torch.cat([x1, x2], dim=1)
        x = self.cls(x)
        return x

In [ ]:
class PetfinderRealDoubleBaseNet(nn.Module):
    # 0149
    def __init__(self):
        super(PetfinderRealDoubleBaseNet, self).__init__()

        self.base_model1 = timm.create_model('tf_efficientnet_b0_ns', num_classes=0, pretrained=False, in_chans=3)
        self.base_model2 = timm.create_model('tf_efficientnet_b0_ns', num_classes=0, pretrained=False, in_chans=3)
        num_features = self.base_model1.num_features

        self.cls = nn.Sequential(
            nn.Linear(num_features * 2, int(num_features)),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(int(num_features), int(num_features / 2)),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(int(num_features / 2), Config.N_LABEL)
        )

    def forward(self, x1, x2):
        x1 = self.base_model1(x1)
        x2 = self.base_model2(x2)
        
        x = torch.cat([x1, x2], dim=1)
        x = self.cls(x)
        return x

In [ ]:
class PetfinderMSENet(nn.Module):
    # for 0087
    def __init__(self, fold):
        super(PetfinderMSENet, self).__init__()
        
        model = PetfinderNet('swin_large_patch4_window12_384')
        
        self.base_model = model.base_model
        num_features = self.base_model.num_features

        self.cls = nn.Sequential(
            nn.Linear(num_features, int(num_features / 2)),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(int(num_features / 2), int(num_features / 4)),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(int(num_features / 4), 1)
        )

    def forward(self, x):
        x = self.base_model(x)
        x = self.cls(x)
        return x

## 10fold models
### 0080

In [ ]:
CP_DIR = Path('../input') / Path('DESKTOP-M3SEAIN_exp0080'.lower().replace('-', '').replace('_', '-'))

In [ ]:
y_preds_0080 = np.zeros(len(test_df))

test_dataset = PetfinderDataSet_base(test_df, test_img_map, get_test_augmentation(IM_SZ=384), data_type='test')
testloader = DataLoader(test_dataset, batch_size=Config.BATCH_SIZE, num_workers=4)

for fold in range(10):

    net = PetfinderNet('swin_large_patch4_window12_384')
    net.to(device)
    print(CP_DIR / f'checkpoint_DESKTOP-M3SEAIN_exp0080_{fold}.pt')
    net.load_state_dict(torch.load(CP_DIR / f'checkpoint_DESKTOP-M3SEAIN_exp0080_{fold}.pt'))

    fold_preds = []
    for i, (inputs, pawpularities) in tqdm(enumerate(testloader), total=len(testloader)):
        net.eval()

        with torch.no_grad():
            
            inputs, pawpularities = inputs.to(device).float(), pawpularities.to(device).reshape(-1, 1).float()
            outputs = net(inputs)
            outputs_np = outputs.sigmoid().to('cpu').detach().numpy().copy().reshape(-1) * 100
            
            print(outputs_np)
            
            fold_preds.append(outputs_np)
    
    y_preds_0080 += np.hstack(fold_preds).reshape(-1) / 10

## 8fold models

## 0158

In [ ]:
CP_DIR = Path('../input') / Path('DESKTOP-M3SEAIN_exp0158'.lower().replace('-', '').replace('_', '-'))

In [ ]:
y_preds_0158 = np.zeros(len(test_df))

test_dataset = PetfinderDataSet_base(test_df, test_img_map, get_test_augmentation(IM_SZ=384), data_type='test')
testloader = DataLoader(test_dataset, batch_size=Config.BATCH_SIZE, num_workers=4)

for fold in range(8):

    net = PetfinderNet('swin_large_patch4_window12_384')
    net.to(device)
    print(CP_DIR / f'checkpoint_DESKTOP-M3SEAIN_exp0158_{fold}.pt')
    net.load_state_dict(torch.load(CP_DIR / f'checkpoint_DESKTOP-M3SEAIN_exp0158_{fold}.pt'))

    fold_preds = []
    for i, (inputs, pawpularities) in tqdm(enumerate(testloader), total=len(testloader)):
        net.eval()

        with torch.no_grad():
            
            inputs, pawpularities = inputs.to(device).float(), pawpularities.to(device).reshape(-1, 1).float()
            outputs = net(inputs)
            outputs_np = outputs.sigmoid().to('cpu').detach().numpy().copy().reshape(-1) * 100
            
            print(outputs_np)
            
            fold_preds.append(outputs_np)
    
    y_preds_0158 += np.hstack(fold_preds).reshape(-1) / 8

## 4 fold models

## c0007, 0030

In [ ]:
CP_DIR1 = Path('../input') / Path('colab3_0007'.lower().replace('-', '').replace('_', '-'))
CP_DIR2 = Path('../input') / Path('DESKTOP-M3SEAIN_exp0030'.lower().replace('-', '').replace('_', '-'))

In [ ]:
y_preds_c0007 = np.zeros(len(test_df))

test_dataset1 = PetfinderDataSet_base(test_df, test_img_map, get_test_augmentation(IM_SZ=384), data_type='test')
testloader1 = DataLoader(test_dataset1, batch_size=Config.BATCH_SIZE, num_workers=4)

for fold in range(4):

    net1 = PetfinderNet('vit_large_r50_s32_384')
    net1.to(device)
    net1.load_state_dict(torch.load(CP_DIR1 / f'checkpoint_colab3_pet_exp0007_{fold}.pt'))

    fold_preds1 = []
    for i, (inputs1, _) in tqdm(enumerate(testloader1), total=len(testloader1)):
        net1.eval()

        with torch.no_grad():
            
            inputs1 = inputs1.to(device).float()
            
            outputs1 = net1(inputs1)
            outputs_np1 = outputs1.sigmoid().to('cpu').detach().numpy().copy().reshape(-1) * 100

            print(outputs_np1)
            
            fold_preds1.append(outputs_np1)
    
    y_preds_c0007 += np.hstack(fold_preds1).reshape(-1) / 4

## 5 fold models

## 0087, 0095, 0097, 0101, 0126, 0149

In [ ]:
CP_DIR1 = Path('../input') / Path('DESKTOP-M3SEAIN_exp0087'.lower().replace('-', '').replace('_', '-'))
#CP_DIR2 = Path('../input') / Path('DESKTOP-M3SEAIN_exp0095'.lower().replace('-', '').replace('_', '-'))
CP_DIR3 = Path('../input') / Path('DESKTOP-M3SEAIN_exp0097'.lower().replace('-', '').replace('_', '-'))
CP_DIR4 = Path('../input') / Path('DESKTOP-M3SEAIN_exp0101'.lower().replace('-', '').replace('_', '-'))
CP_DIR5 = Path('../input') / Path('DESKTOP-M3SEAIN_exp0126'.lower().replace('-', '').replace('_', '-'))
CP_DIR6 = Path('../input') / Path('DESKTOP-M3SEAIN_exp0149'.lower().replace('-', '').replace('_', '-'))

In [ ]:
y_preds_0087 = np.zeros(len(test_df))
#y_preds_0095 = np.zeros(len(test_df))
y_preds_0097 = np.zeros(len(test_df))
y_preds_0101 = np.zeros(len(test_df))
y_preds_0126 = np.zeros(len(test_df))
y_preds_0149 = np.zeros(len(test_df))

test_dataset_384 = PetfinderDataSet_base(test_df, test_img_map, get_test_augmentation(IM_SZ=384), data_type='test')
testloader_384 = DataLoader(test_dataset_384, batch_size=Config.BATCH_SIZE//2, num_workers=4)

test_dataset_fll_256 = PetfinderDataSet_fllsize(test_df, test_img_map, get_test_augmentation(IM_SZ=256), data_type='test', fllsize_transforms=get_fllsize_augmentation(IM_SZ=256))
testloader_fll_256 = DataLoader(test_dataset_fll_256, batch_size=Config.BATCH_SIZE//2, num_workers=4)

test_dataset_224 = PetfinderDataSet_base(test_df, test_img_map, get_test_augmentation(IM_SZ=224), data_type='test')
testloader_224 = DataLoader(test_dataset_224, batch_size=Config.BATCH_SIZE//2, num_workers=4)

test_dataset_fll_300 = PetfinderDataSet_fllsize(test_df, test_img_map, get_test_augmentation(IM_SZ=300), data_type='test', fllsize_transforms=get_fllsize_augmentation(IM_SZ=300))
testloader_fll_300 = DataLoader(test_dataset_fll_300, batch_size=Config.BATCH_SIZE//2, num_workers=4)

for fold in range(5):

    #net1 = PetfinderMSENet(fold)
    #net1.to(device)
    #net1.load_state_dict(torch.load(CP_DIR1 / f'checkpoint_DESKTOP-M3SEAIN_exp0087_{fold}.pt'))

    #net2 = PetfinderDoubleBaseNet()
    #net2.to(device)
    #net2.load_state_dict(torch.load(CP_DIR2 / f'checkpoint_DESKTOP-M3SEAIN_exp0095_{fold}.pt'))
  
    #net3 = PetfinderNet('resnet18d')
    #net3.to(device)
    #net3.load_state_dict(torch.load(CP_DIR3 / f'checkpoint_DESKTOP-M3SEAIN_exp0097_{fold}.pt'))

    net4 = PetfinderNet('swin_large_patch4_window12_384')
    net4.to(device)
    net4.load_state_dict(torch.load(CP_DIR4 / f'checkpoint_DESKTOP-M3SEAIN_exp0101_{fold}.pt'))
    
    net5 = PetfinderNet('densenet121')
    net5.to(device)
    net5.load_state_dict(torch.load(CP_DIR5 / f'checkpoint_DESKTOP-M3SEAIN_exp0126_{fold}.pt'))
    
    net6 = PetfinderRealDoubleBaseNet()
    net6.to(device)
    net6.load_state_dict(torch.load(CP_DIR6 / f'checkpoint_DESKTOP-M3SEAIN_exp0149_{fold}.pt'))
    
    fold_preds1 = []
    #fold_preds2 = []
    fold_preds3 = []
    fold_preds4 = []
    fold_preds5 = []
    fold_preds6 = []
    #for i, ((inputs384, _), (inputs256, full_img256, _), (inputs224, _), (inputs300, full_img300, _)) in tqdm(enumerate(zip(testloader_384, testloader_fll_256, testloader_224, testloader_fll_300)), total=len(testloader_384)):
    for i, ((inputs384, _), (inputs224, _), (inputs300, full_img300, _)) in tqdm(enumerate(zip(testloader_384, testloader_224, testloader_fll_300)), total=len(testloader_384)):
        #net1.eval()
        #net2.eval()
        #net3.eval()
        net4.eval()
        net5.eval()
        net6.eval()

        with torch.no_grad():
            
            inputs384 = inputs384.to(device).float()
            
            #outputs1 = net1(inputs384)
            #outputs_np1 = outputs1.to('cpu').detach().numpy().copy().reshape(-1)
            
            #inputs256, full_img256 = inputs256.to(device).float(), full_img256.to(device).float()
            #outputs2 = net2(inputs256, full_img256)
            #outputs_np2 = outputs2.to('cpu').detach().numpy().copy().reshape(-1)
         
            inputs224 = inputs224.to(device).float()
            #outputs3 = net3(inputs224)
            #outputs_np3 = outputs3.sigmoid().to('cpu').detach().numpy().copy().reshape(-1) * 100    
    
            outputs4 = net4(inputs384)
            outputs_np4 = outputs4.sigmoid().to('cpu').detach().numpy().copy().reshape(-1) * 100
    
            outputs5 = net5(inputs224)
            outputs_np5 = outputs5.sigmoid().to('cpu').detach().numpy().copy().reshape(-1) * 100
    
            inputs300, full_img300 = inputs300.to(device).float(), full_img300.to(device).float()
            outputs6 = net6(inputs300, full_img300)
            outputs_np6 = outputs6.to('cpu').detach().numpy().copy().reshape(-1)
    
            #fold_preds1.append(outputs_np1)
            #fold_preds2.append(outputs_np2)
            #fold_preds3.append(outputs_np3)
            fold_preds4.append(outputs_np4)
            fold_preds5.append(outputs_np5)
            fold_preds6.append(outputs_np6)
    
    #y_preds_0087 += np.hstack(fold_preds1).reshape(-1) / 5
    #y_preds_0095 += np.hstack(fold_preds2).reshape(-1) / 5
    #y_preds_0097 += np.hstack(fold_preds3).reshape(-1) / 5
    y_preds_0101 += np.hstack(fold_preds4).reshape(-1) / 5
    y_preds_0126 += np.hstack(fold_preds5).reshape(-1) / 5
    y_preds_0149 += np.hstack(fold_preds6).reshape(-1) / 5

## 前回コンペと同じ画像抽出

In [ ]:
base_model = timm.create_model('efficientnet_b0', num_classes=0, pretrained=False, in_chans=3)
base_model.load_state_dict(torch.load('../input/efficientnet-b0/efficientnet_b0.pt'))
base_model = base_model.to(device)

In [ ]:
org_test_img_paths = glob.glob(str(DATA_DIR /  'test/*'))
#pre_train_img_paths = glob.glob('../input/prepetfinder/pre_petfinder/train_images/*')
pre_img_paths = glob.glob('../input/prepetfinder/pre_petfinder/train_images/*') + glob.glob('../input/prepetfinder/pre_petfinder_test/test_images/*')

In [ ]:
def get_augmentation():
    train_transform = [
        albu.Resize(height=224, width=224)
    ]
    return albu.Compose(train_transform)

class DataSet(Dataset):
    def __init__(self, image_paths):
        self.image_paths = image_paths
        self.transforms = get_augmentation()

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = cv2.imread(img_path)
        img = self.transforms(image=img)['image']
        img = np.moveaxis(img, 2, 0)

        return img


In [ ]:
#pre_train_vectors = np.load('../input/pet2-sim-tensor-calc-challenge/pre_train_vectors.npy')
pre_vectors = np.load('../input/pre-pet-vectors/pre_pet_vectors.npy')

In [ ]:
dataset = DataSet(org_test_img_paths)
loader = DataLoader(dataset, batch_size=64, shuffle=False, drop_last=False, num_workers=4)

outputs = []
for img in tqdm(loader, total=len(loader)):
    img = img.to(device).float()
    output = base_model(img)
    outputs_np = output.to('cpu').detach().numpy().copy()
    outputs.append(outputs_np)

org_test_vectors = np.vstack(outputs)

### 類似度算出

In [ ]:
def cos_sim_matrix(matrix1, matrix2):
    """
    item-feature 行列が与えられた際に
    item 間コサイン類似度行列を求める関数
    """
    # d = matrix @ matrix.T  # item-vector 同士の内積を要素とする行列
    # matrix1 : M * V, matrix2 : V * N
    d = matrix1 @ matrix2

    # コサイン類似度の分母に入れるための、各 item-vector の大きさの平方根
    # V の次元で sum : (M * V)
    norm1 = (matrix1 * matrix1).sum(axis=1, keepdims=True) ** .5
    # V の次元で sum : (V * N)
    norm2 = (matrix2 * matrix2).sum(axis=0, keepdims=True) ** .5

    # それぞれの item の大きさの平方根で割っている（なんだかスマート！）
    return d / norm1 / norm2

In [ ]:
%%time

# 行列演算で test_img_num * pre_train_img_num の類似度行列を獲得する
d = cos_sim_matrix(org_test_vectors, pre_vectors.T)
print(d.shape)
print(d)

# argmax で test_img と最も類似している pre_train_img が,
# max でその類似度を獲得できる
print(np.argmax(d, 1), np.max(d, 1))

# ↑ これらを対応させた辞書にして完成
img_pair = [
    (org_test, pre_img_paths[sim_argmax], sim_max) for org_test, sim_argmax, sim_max in zip(org_test_img_paths, np.argmax(d, 1), np.max(d, 1))
]

img_pair

## kaeru

### 42, 44

In [ ]:
import albumentations as A
import cv2
import gc
import os
import math
import random
import time
import warnings
import sys

import numpy as np
import pandas as pd
import transformers
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.optim import Adam, SGD
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from pathlib import Path
from typing import List
from PIL import Image

from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.model_selection import KFold,StratifiedKFold

from tqdm import tqdm

In [ ]:
class CFG:
    ######################
    # Globals #
    ######################
    seed = 71
    epochs = 20
    folds = [0, 1, 2, 3, 4]
    N_FOLDS = 5
    LR = 1e-4
    train_bs = 16
    valid_bs = 32
    train_root = '../input/petfinder-pawpularity-score/train/'
    test_root = '../input/petfinder-pawpularity-score/test/'
    in_chans = 3
    ID_COL = 'Id'
    TARGET_COL = 'Pawpularity'
    TARGET_DIM = 1
    FEATURE_COLS = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action',
        'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 
        'Info', 'Blur',
    ]

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
class Pet2Dataset:
    def __init__(self, X, y=None, Meta_features=None, transforms=None):
        self.X = X
        self.y = y
        self.Meta_features = Meta_features
        self.transforms = transforms

    def __len__(self):
        return len(self.X)

    def __getitem__(self, item):
        path = CFG.test_root + self.X[item] + '.jpg'
        features = cv2.imread(path)
        features = cv2.cvtColor(features, cv2.COLOR_BGR2RGB)
        features = self.transforms['valid'](image=features)['image']
        features = np.transpose(features, (2, 0, 1)).astype(np.float32)

        return {
            'x': torch.tensor(features, dtype=torch.float32),
            'meta': torch.tensor(self.Meta_features[item], dtype=torch.float32),
        }

In [ ]:
class Pet2Model(nn.Module):
    def __init__(self, model_name):
        super(Pet2Model, self).__init__()    
        
        # Model Encoder
        self.model = timm.create_model(model_name, pretrained=False, num_classes=0, in_chans=CFG.in_chans)
        self.model.head = nn.Linear(self.model.num_features, 128)
        self.dense = nn.Linear(128, CFG.TARGET_DIM)

    def forward(self, features):
        x = self.model(features)
        output = self.dense(x)
        return output.squeeze(-1)

In [ ]:
class Pet2CNNModel(nn.Module):
    def __init__(self, model_name):
        super(Pet2CNNModel, self).__init__()    
        
        # Model Encoder
        self.model = timm.create_model(model_name, pretrained=False, num_classes=0, in_chans=CFG.in_chans)
        self.model.fc = nn.Linear(self.model.num_features, 128)
        self.dense = nn.Linear(128, CFG.TARGET_DIM)

    def forward(self, features):
        x = self.model(features)
        output = self.dense(x)
        return output.squeeze(-1)

In [ ]:
class Pet2VitCNNModel(nn.Module):
    def __init__(self, model_name):
        super(Pet2VitCNNModel, self).__init__()    
        
        # Model Encoder
        self.model = timm.create_model(model_name, pretrained=False, num_classes=0, in_chans=CFG.in_chans)
        self.model.head = nn.Linear(self.model.num_features, 128)
        self.dense = nn.Linear(128, CFG.TARGET_DIM)

    def forward(self, features):
        x = self.model(features)
        output = self.dense(x)
        return output.squeeze(-1)

In [ ]:
def make_preds(model_dict1, model_dict2, model_dict3, model_dict4, model_dict5):    
    df = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
    y_pred1 = []
    y_pred2 = []
    y_pred3 = []
    y_pred4 = []
    y_pred5 = []
    for fold in CFG.folds:
        model1 = Pet2Model(model_dict1['MODEL_NAME'])
        model1.to(device)
        model1.load_state_dict(torch.load(f'../input/kaerururu-petfinder2-{model_dict1["EXP_ID"]}/fold-{fold}.bin'))
        model1.eval()
        
        model2 = Pet2Model(model_dict2['MODEL_NAME'])
        model2.to(device)
        model2.load_state_dict(torch.load(f'../input/kaerururu-petfinder2-{model_dict2["EXP_ID"]}/fold-{fold}.bin'))
        model2.eval()
        
        model3 = Pet2Model(model_dict3['MODEL_NAME'])
        model3.to(device)
        model3.load_state_dict(torch.load(f'../input/kaerururu-petfinder2-{model_dict3["EXP_ID"]}/fold-{fold}.bin'))
        model3.eval()
        
        model4 = Pet2CNNModel(model_dict4['MODEL_NAME'])
        model4.to(device)
        model4.load_state_dict(torch.load(f'../input/kaerururu-petfinder2-{model_dict4["EXP_ID"]}/fold-{fold}.bin'))
        model4.eval()
        
        model5 = Pet2VitCNNModel(model_dict5['MODEL_NAME'])
        model5.to(device)
        model5.load_state_dict(torch.load(f'../input/kaerururu-petfinder2-{model_dict5["EXP_ID"]}/fold-{fold}.bin'))
        model5.eval()
        
        dataset1 = Pet2Dataset(X=df[CFG.ID_COL].values, y=None, Meta_features=df[CFG.FEATURE_COLS].values, transforms=model_dict1['TRANSFORM'])
        data_loader1 = torch.utils.data.DataLoader(
            dataset1, batch_size=CFG.valid_bs//4, num_workers=0, pin_memory=True, shuffle=False
        )
        
        dataset2 = Pet2Dataset(X=df[CFG.ID_COL].values, y=None, Meta_features=df[CFG.FEATURE_COLS].values, transforms=model_dict2['TRANSFORM'])
        data_loader2 = torch.utils.data.DataLoader(
            dataset2, batch_size=CFG.valid_bs//4, num_workers=0, pin_memory=True, shuffle=False
        )
        
        del dataset1, dataset2; gc.collect()

        final_output1 = []
        final_output2 = []
        final_output3 = []
        final_output4 = []
        final_output5 = []
        for b_idx, (data1, data2) in tqdm(enumerate(zip(data_loader1, data_loader2))):
            with torch.no_grad():
                inputs384 = data1['x'].to(device)
                inputs224 = data2['x'].to(device)
                                          
                output1 = model1(inputs384)
                output2 = model2(inputs224)
                output3 = model3(inputs224)
                output4 = model4(inputs384)
                output5 = model5(inputs384)
                
                output1 = torch.sigmoid(output1) * 100.
                output1 = output1.detach().cpu().numpy().tolist()
                final_output1.extend(output1)
                
                output2 = torch.sigmoid(output2) * 100.
                output2 = output2.detach().cpu().numpy().tolist()
                final_output2.extend(output2)
                
                output3 = torch.sigmoid(output3) * 100.
                output3 = output3.detach().cpu().numpy().tolist()
                final_output3.extend(output3)
                
                output4 = torch.sigmoid(output4) * 100.
                output4 = output4.detach().cpu().numpy().tolist()
                final_output4.extend(output4)
                
                output5 = torch.sigmoid(output5) * 100.
                output5 = output5.detach().cpu().numpy().tolist()
                final_output5.extend(output5)
                
        y_pred1.append(np.array(final_output1))
        y_pred2.append(np.array(final_output2))
        y_pred3.append(np.array(final_output3))
        y_pred4.append(np.array(final_output4))
        y_pred5.append(np.array(final_output5))
        torch.cuda.empty_cache()
        
    y_pred1 = np.mean(y_pred1, 0)
    y_pred2 = np.mean(y_pred2, 0)
    y_pred3 = np.mean(y_pred3, 0)
    y_pred4 = np.mean(y_pred4, 0)
    y_pred5 = np.mean(y_pred5, 0)
    
    del model1, model2, model3, model4, model5; gc.collect()
    del final_output1, final_output2, final_output3, final_output4, final_output5; gc.collect()
    return y_pred1, y_pred2, y_pred3, y_pred4, y_pred5

In [ ]:
# environment
set_seed(CFG.seed)

model_dict1 = {
    'EXP_ID' : '042', 
    'MODEL_NAME' : 'swin_large_patch4_window12_384',
    'TRANSFORM' : {
        'valid' : A.Compose([
            A.Resize(384, 384, p=1),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0,),
        ], p=1.0),    
    }
}

model_dict2 = {
    'EXP_ID' : '044', 
    'MODEL_NAME' : 'swin_large_patch4_window7_224',
    'TRANSFORM' : {
        'valid' : A.Compose([
            A.Resize(224, 224, p=1),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0,),
        ], p=1.0),    
    }
}

model_dict3 = {
    'EXP_ID' : '055', 
    'MODEL_NAME' : 'swin_large_patch4_window7_224',
    'TRANSFORM' : {
        'valid' : A.Compose([
            A.Resize(224, 224, p=1),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0,),
        ], p=1.0),    
    }
}

model_dict4 = {
    'EXP_ID' : '084', 
    'MODEL_NAME' : 'resnext101_32x8d',
    'TRANSFORM' : {
        'valid' : A.Compose([
            A.Resize(384, 384, p=1),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0,),
        ], p=1.0),    
    }
}

model_dict5 = {
    'EXP_ID' : '062', 
    'MODEL_NAME' : 'vit_base_r50_s16_384',
    'TRANSFORM' : {
        'valid' : A.Compose([
            A.Resize(384, 384, p=1),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0,),
        ], p=1.0),    
    }
}

sub = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
preds_042, preds_044, preds_055, preds_084, preds_062 = make_preds(model_dict1, model_dict2, model_dict3, model_dict4, model_dict5)

In [ ]:
pre_sim_df = pd.DataFrame(img_pair, columns=['org_path', 'pre_path', 'sim'])
pre_sim_df

In [ ]:
#pre_train_df = pd.read_csv('../input/prepetfinder/pre_petfinder/train/train.csv')
# pre_df = pd.concat([pd.read_csv('../input/prepetfinder/pre_petfinder/train/train.csv'), pd.read_csv('../input/prepetfinder/pre_petfinder_test/test/test.csv')])

pre_df = pd.read_csv('../input/pp-stacking-avg-v019-nlp-feature/prepetfinder_add_emoji.csv')
pre_df.head(2)

In [ ]:
pre_sim_list = []
for idx, (org_path, pre_path, sim) in tqdm(enumerate(zip(pre_sim_df['org_path'].values, pre_sim_df['pre_path'].values, pre_sim_df['sim'].values))): 
    org_id = org_path.split('/')[-1].split('.')[0]
    pre_id = pre_path.split('/')[-1].split('-')[0]
    
    adoptionSpeed = pre_df[pre_df['PetID'] == pre_id].iloc[0]['AdoptionSpeed']
    
    pre_sim_list.append(
        {
            'org_path':org_path,
            'pre_path':pre_path,
            'sim':sim,
            'org_id':org_id,
            'pre_id':pre_id,
            'adoptionSpeed':adoptionSpeed
            
        }
    )
pwup_speed_df = pd.DataFrame(pre_sim_list)
pwup_speed_df

In [ ]:
test_df = test_df.merge(pwup_speed_df[['org_id', 'adoptionSpeed', 'sim', 'pre_id']], how='left', left_on='Id', right_on='org_id').reset_index()
display(test_df)
# test_df = test_df.merge(pd.concat([pd.read_csv('../input/prepetfinder/pre_petfinder/train/train.csv'), pd.read_csv('../input/prepetfinder/pre_petfinder_test/test/test.csv')]), how='left', left_on='pre_id', right_on='PetID')
test_df = test_df.merge(pd.read_csv('../input/pp-stacking-avg-v019-nlp-feature/prepetfinder_add_emoji.csv'), how='left', left_on='pre_id', right_on='PetID')

test_df['Breed_205'] = (test_df['Breed1'] == 205).astype(int)
test_df['Breed_307'] = (test_df['Breed1'] == 307).astype(int)
# test_df['Breed_266'] = (test_df['Breed1'] == 266).astype(int)
test_df['Breed_265'] = (test_df['Breed1'] == 265).astype(int)

# test_df['Breed_299'] = (test_df['Breed1'] == 299).astype(int)
test_df['Breed_264'] = (test_df['Breed1'] == 264).astype(int)
# test_df['Breed_292'] = (test_df['Breed1'] == 292).astype(int)

test_df['Breed_285'] = (test_df['Breed1'] == 285).astype(int)
test_df['Breed_141'] = (test_df['Breed1'] == 141).astype(int)
test_df['Breed_218'] = (test_df['Breed1'] == 218).astype(int)
test_df['Breed_205'] = (test_df['Breed1'] == 205).astype(int)

# test_df['Breed_179'] = (test_df['Breed1'] == 179).astype(int)
test_df['Breed_103'] = (test_df['Breed1'] == 103).astype(int)

test_df['Breed_266_265_299'] = test_df['Breed1'].map(lambda x: 1 if x in [266, 265, 299] else 0)
test_df['Breed_292_179'] = test_df['Breed1'].map(lambda x: 1 if x in [292, 179] else 0)

test_df['RescuerID_1'] = (test_df['RescuerID'] == 'fa90fa5b1ee11c86938398b60abc32cb').astype(int)
test_df['RescuerID_2'] = (test_df['RescuerID'] == 'aa66486163b6cbc25ea62a34b11c9b91').astype(int)
test_df['RescuerID_3'] = (test_df['RescuerID'] == 'c00756f2bdd8fa88fc9f07a8309f7d5d').astype(int)
test_df['RescuerID_4'] = (test_df['RescuerID'] == 'b53c34474d9e24574bcec6a3d3306a0d').astype(int)
test_df['RescuerID_5'] = (test_df['RescuerID'] == 'b770bac0ca797cf1433c48a35d30c4cb').astype(int)

test_df['Name_word_len'] = test_df['Name'].fillna('').apply(lambda x : len(x.split(' ')))
test_df['Description_len'] = test_df['Description'].fillna('').apply(lambda x : len(x.split(' ')))


In [ ]:
# st_models = unpickle('../input/fork-of-pp-stacking-avg-v016-reduce-pp-features2/pp_models_stacking.pkl')
st_models = unpickle('../input/defense-pp-stacking-avg-v034/pp_models_stacking.pkl')

In [ ]:
test_df['y_preds_0126'] = y_preds_0126
test_df['y_preds_0101'] = y_preds_0101
#test_df['y_preds_0095'] = y_preds_0095
test_df['y_preds_0080'] = y_preds_0080
test_df['y_preds_0158'] = y_preds_0158
test_df['y_preds_c0007'] = y_preds_c0007
test_df['y_johannyjm_1790651'] = y_johannyjm_1790651
test_df['preds_042'] = preds_042
test_df['preds_044'] = preds_044
test_df['preds_055'] = preds_055
# test_df['preds_060'] = preds_060
test_df['preds_084'] = preds_084
# test_df['preds_085'] = preds_085
test_df['preds_062'] = preds_062
# test_df['preds_074'] = preds_074
test_df['y_preds_0149'] = y_preds_0149

test_df['fast_ai_004_preds'] = fast_ai_004_preds

In [ ]:
#test_df['sim'] = 1
#test_df['adoptionSpeed'] = 0

## Stacking

In [ ]:
pp_stacking_dfs = []

for c, model_as in enumerate(st_models):
    
    pred_stack = model_as[0].predict(test_df[['y_preds_0158', 'y_preds_0149', 'y_preds_0080', 'y_preds_c0007', 'y_johannyjm_1790651', 
                                              'preds_042', 'preds_044', 'preds_055', 'fast_ai_004_preds', 
                                              'preds_084', 'preds_062', 'y_preds_0126', 
                                              # 'preds_074'
                                             ]])
    
    test_df['Pawpularity'] = pred_stack

    under_threshold_df = test_df[test_df['sim'] < 0.65].copy()
    over_threshold_df = test_df[test_df['sim'] >= 0.65].copy()
    
    if len(over_threshold_df) > 0:
        over_threshold_df['Pawpularity'] = model_as[2].predict(over_threshold_df[['Pawpularity', 'Breed_307', 'Breed_266_265_299', 'Breed_292_179', 'Breed_285', 'Breed_103', 'Age', 'RescuerID_3', 'RescuerID_5', 'Description_len', 'PhotoAmt', 'Name_word_len', 'is_emoji']])
        print('over_threshold_df')
        
    over_threshold_df_as_over0 = over_threshold_df[over_threshold_df['adoptionSpeed'] >= 0].copy()
    over_threshold_df_other = over_threshold_df[~(over_threshold_df['adoptionSpeed'].isin([i for i in range(5)]))].copy()
        
    if len(over_threshold_df_as_over0) > 0:
        over_threshold_df_as_over0['Pawpularity'] = model_as[1].predict(over_threshold_df_as_over0[['Pawpularity', 'Breed_205', 'Breed_307', 'Breed_266_265_299', 'Age', 'adoptionSpeed']])
        print('over_threshold_df_as_over0')
        
    test_adjust_df = pd.concat([under_threshold_df, over_threshold_df_as_over0, over_threshold_df_other]).sort_values('index')

    pp_stacking_dfs.append(test_adjust_df.copy())

In [ ]:
pp_stacking_df = pd.concat(pp_stacking_dfs)
pp_stacking_df.head(2)

## Average

In [ ]:
# avg_models = unpickle('../input/fork-of-pp-stacking-avg-v016-reduce-pp-features2/pp_models_avg.pkl')
avg_models = unpickle('../input/defense-pp-stacking-avg-v034/pp_models_avg.pkl')

In [ ]:
pp_avg_dfs = []

for c, model_as in enumerate(avg_models):
    
    pred_avg =  test_df[['y_preds_0158', 'y_preds_0149', 'y_preds_0080', 'y_preds_c0007', 'y_johannyjm_1790651', 
                         'preds_042', 
                         'preds_044', # 'preds_085',
                         'preds_055', 'fast_ai_004_preds', 
                         'preds_084', 'preds_062', 'y_preds_0126', 
                         # 'preds_074'
                        ]].mean(axis=1)
    
    test_df['Pawpularity'] = pred_avg

    under_threshold_df = test_df[test_df['sim'] < 0.65].copy()
    over_threshold_df = test_df[test_df['sim'] >= 0.65].copy()
    
    if len(over_threshold_df) > 0:
        over_threshold_df['Pawpularity'] = model_as[2].predict(over_threshold_df[['Pawpularity', 'Breed_307', 'Breed_266_265_299', 'Breed_292_179', 'Breed_285', 'Breed_103', 'Age', 'RescuerID_3', 'RescuerID_5', 'Description_len', 'PhotoAmt', 'Name_word_len', 'is_emoji']])
        print('over_threshold_df')
    
    over_threshold_df_as_over0 = over_threshold_df[over_threshold_df['adoptionSpeed'] >= 0].copy()
    over_threshold_df_other = over_threshold_df[~(over_threshold_df['adoptionSpeed'].isin([i for i in range(5)]))].copy()
    
    if len(over_threshold_df_as_over0) > 0:
        over_threshold_df_as_over0['Pawpularity'] = model_as[1].predict(over_threshold_df_as_over0[['Pawpularity', 'Breed_205', 'Breed_307', 'Breed_266_265_299', 'Age', 'adoptionSpeed']])
        print('over_threshold_df_as_over0')
        
    test_adjust_df = pd.concat([under_threshold_df, over_threshold_df_as_over0, over_threshold_df_other]).sort_values('index')

    pp_avg_dfs.append(test_adjust_df.copy())

In [ ]:
pp_avg_df = pd.concat(pp_avg_dfs)
pp_avg_df.head(2)

In [ ]:
pp_avg_mean_df = pp_avg_df.groupby('Id')['Pawpularity'].mean().to_frame()

In [ ]:
pp_stacking_mean_df = pp_stacking_df.groupby('Id')['Pawpularity'].mean().to_frame()

In [ ]:
pp_df = pd.concat([pp_avg_mean_df, pp_stacking_mean_df])
pp_df

In [ ]:
pp_df = pp_df.groupby('Id')['Pawpularity'].mean().to_frame().reset_index()
pp_df

In [ ]:
# y_preds_0101

In [ ]:
y_preds_0158

In [ ]:
y_preds_c0007

In [ ]:
# y_preds_0030

In [ ]:
#y_preds_0095

In [ ]:
y_johannyjm_1790651

In [ ]:
preds_042

In [ ]:
preds_044

In [ ]:
preds_055

In [ ]:
preds_084 # preds_060

In [ ]:
# preds_085

In [ ]:
fast_ai_004_preds

In [ ]:
preds_062

In [ ]:
y_preds_0126

In [ ]:
# preds_074

In [ ]:
y_preds_0149

In [ ]:
y_preds_0158

In [ ]:
y_preds_0080

In [ ]:
submission_df = pp_df[['Id', 'Pawpularity']]
submission_df.to_csv('submission.csv', index=False)
submission_df